## Threading methods

This is a simple example to show how threading methods work. The program uses these methods and checks if any errors occur. 

In [ ]:
?MPI::MPI_Init_thread

In [ ]:
?MPI::MPI_Query_thread

In [ ]:
?MPI::MPI_Is_thread_main

***
#### C example

In [ ]:
#include <mpi.h>

In [ ]:
%%executable  a.x -- -lmpi

int errs = 0;
int provided, flag, claimed;

MPI_Init_thread(NULL, NULL, MPI_THREAD_MULTIPLE, &provided);

MPI_Is_thread_main(&flag);
if (!flag) {
    errs++;
    printf("This thread called init_thread but Is_thread_main gave false\n");
}

MPI_Query_thread(&claimed);
if (claimed != provided) {
    errs++;
    printf("Query thread gave thread level %d but Init_thread gave %d\n", claimed, provided );
}

printf("Errors: %d ", errs);

MPI_Finalize();

In [ ]:
!mpirun -np 2 a.x

***
#### Python example

`MPI_Init()` or `MPI_Init_thread()` is actually called when you import the MPI module from the mpi4py package, if MPI is not already initialized. In such case, calling the methods again from Python is expected to generate and MPI error. 

If `thread_level` is set to `True`, the init is done when `from mpi4py import MPI`. If `thread_level` is set to `False`, it is not done so we need to write the init. 

In [ ]:
%%file threading.py
from mpi4py import rc
rc.initialize = False
rc.thread_level = 'multiple'

from mpi4py import MPI

MPI.Init()

errs = 0
flag = MPI.Is_thread_main()
if not flag:
    errs += 1
    print("This thread called init_thread but Is_thread_main gave false")

claimed = MPI.Query_thread()

MPI.Finalize()

In [ ]:
!mpirun -np 2 python threading.py

***
#### Fortran example

In [ ]:
%%file threading.f90
program threading
use mpi

integer ( kind = 4 ) error
integer :: errs, provided, claimed
logical :: flag
errs = 0

call MPI_Init_thread(MPI_THREAD_MULTIPLE, provided, error)

call MPI_Is_thread_main(flag, error)
if (flag .eqv. .false.) then
    errs = errs + 1
    print *, "This thread called init_thread but Is_thread_main gave false"
end if

call MPI_Query_thread(claimed, error)
if (claimed .ne. provided) then
    errs = errs + 1
    print *, "Query thread gave thread level ", claimed, "but Init_thread gave ", provided 
end if

print *, "Errors: ", errs

call MPI_Finalize(error)
end

In [ ]:
!mpif90 threading.f90 && mpirun -np 2 a.out